In [1]:
# !pip install https://github.com/JamesRitchie/scikit-rvm/archive/master.zip

In [2]:
import numpy as np
import pandas as pd 
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skrvm import RVC
import extract_vector as ev

In [3]:
arousal_dataset = pickle.load(open("../Dataset/FusedData/FusedAllData.pkl","rb"))
arousal_label = ev.getLabelData(type="arousal")

valence_dataset = pickle.load(open("../Dataset/FusedData/FusedAllData.pkl","rb"))
valence_label = ev.getLabelData(type="valence")

In [4]:
arousal_dataset = arousal_dataset.astype(float)
arousal_label = arousal_label.astype(float)

valence_dataset = valence_dataset.astype(float)
valence_label = valence_label.astype(float)

In [5]:
def getLabel(label):
    new_label=[]
    for i in range(len(label)):
        if(label[i][0]>=4.5):
            new_label.append(1)
        else:
            new_label.append(0)
    return new_label

In [6]:
# clean data
def clean_data(dataset, target):
    del_rows = []
    for i in range(len(dataset)):
        if(np.isnan(dataset[i]).sum() > 0):
            del_rows.append(i)
            
    dataset = np.delete(dataset, del_rows, axis=0)
    target = np.delete(target, del_rows, axis=0)
    
    return dataset,target

In [7]:
arousal_dataset,arousal_label = clean_data(arousal_dataset,arousal_label)
valence_dataset,valence_label = clean_data(valence_dataset,valence_label)

In [8]:
print(arousal_dataset.shape)

(755, 126)


In [9]:
sc = StandardScaler()

arousal_dataset = sc.fit_transform(arousal_dataset)
valence_dataset = sc.fit_transform(valence_dataset)

In [10]:
arousal_label = getLabel(arousal_label)
valence_label = getLabel(valence_label)

In [11]:
pca = PCA(n_components=0.95)

arousal_dataset = pca.fit_transform(arousal_dataset)
valence_dataset = pca.fit_transform(valence_dataset)

In [12]:
ar_model = RVC()
val_model = RVC()

In [13]:
x_a_train,x_a_test,y_a_train,y_a_test = train_test_split(arousal_dataset,arousal_label,test_size=0.2,random_state=42)
x_v_train,x_v_test,y_v_train,y_v_test = train_test_split(valence_dataset,valence_label,test_size=0.2,random_state=42)

In [14]:
ar_model.fit(x_a_train,np.array(y_a_train))

RVC(alpha=1e-06, beta=1e-06, beta_fixed=False, bias_used=True, coef0=0.0,
    coef1=None, degree=3, kernel='rbf', n_iter=3000,
    threshold_alpha=1000000000.0, tol=0.001, verbose=False)

In [15]:
pred_a = ar_model.predict(x_a_train)
print(classification_report(y_a_train,pred_a))

              precision    recall  f1-score   support

           0       0.94      0.78      0.85       185
           1       0.91      0.98      0.94       419

    accuracy                           0.92       604
   macro avg       0.92      0.88      0.90       604
weighted avg       0.92      0.92      0.91       604



In [16]:
pred_a = ar_model.predict(x_a_test)
print(classification_report(y_a_test,pred_a))

              precision    recall  f1-score   support

           0       0.57      0.25      0.35        52
           1       0.70      0.90      0.78        99

    accuracy                           0.68       151
   macro avg       0.63      0.57      0.57       151
weighted avg       0.65      0.68      0.63       151



In [17]:
val_model.fit(x_v_train,np.array(y_v_train))

RVC(alpha=1e-06, beta=1e-06, beta_fixed=False, bias_used=False, coef0=0.0,
    coef1=None, degree=3, kernel='rbf', n_iter=3000,
    threshold_alpha=1000000000.0, tol=0.001, verbose=False)

In [18]:
pred_v = val_model.predict(x_v_train)
print(classification_report(y_v_train,pred_v))

              precision    recall  f1-score   support

           0       0.79      0.71      0.75       243
           1       0.82      0.88      0.84       361

    accuracy                           0.81       604
   macro avg       0.80      0.79      0.80       604
weighted avg       0.81      0.81      0.81       604



In [19]:
pred_v = val_model.predict(x_v_test)
print(classification_report(y_v_test,pred_v))

              precision    recall  f1-score   support

           0       0.40      0.32      0.35        60
           1       0.60      0.68      0.64        91

    accuracy                           0.54       151
   macro avg       0.50      0.50      0.50       151
weighted avg       0.52      0.54      0.53       151

